# Comandos para realização do trabalho da matéria de Big Data com uso da biblioteca PySpark.

Este notebook foi projetado para guiar os alunos na realização das práticas de Big Data utilizando PySpark. Certifique-se de seguir cada etapa cuidadosamente para garantir a correta execução das atividades.

Seu trabalho começará na célula 5. Execute as 4 primeiras células para iniciar a atividade.

## <font color=red>Observação importante:</font>

<font color=yellow>Trabalho realizado com uso da biblioteca pandas não será aceito!</font>

## Upload do arquivo `imdb-reviews-pt-br.csv` para dentro do Google Colab

Aqui, você fará o download do dataset necessário para as atividades. Certifique-se de que o arquivo foi descompactado corretamente antes de prosseguir.

In [ ]:
!wget https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip -O imdb-reviews-pt-br.zip
!unzip imdb-reviews-pt-br.zip
!rm imdb-reviews-pt-br.zip

--2025-09-06 02:28:56--  https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49549692 (47M) [application/zip]
Saving to: ‘imdb-reviews-pt-br.zip’

imdb-reviews-pt-br. 100%[===================>]  47.25M   199MB/s    in 0.2s    

2025-09-06 02:28:57 (199 MB/s) - ‘imdb-reviews-pt-br.zip’ saved [49549692/49549692]

Archive:  imdb-reviews-pt-br.zip
replace imdb-reviews-pt-br.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Instalação manual das dependências para uso do pyspark no Google Colab

Esta etapa garante que todas as bibliotecas necessárias para o PySpark sejam instaladas no Google Colab.

In [ ]:
!pip install pyspark

## Importar, instanciar e criar a SparkSession

A SparkSession é o ponto de entrada para usar o PySpark. Certifique-se de configurar corretamente o nome do aplicativo e o master.

In [ ]:
from pyspark.sql import SparkSession

appName = "PySpark Trabalho de Big Data"
master = "local"

spark = SparkSession.builder.appName(appName).master(master).getOrCreate()

## Criar spark dataframe do CSV utilizando o método read.csv do spark

Não altere este código e use o dataframe imdb_df criado aqui em todo o seu trabalho. A criação de um dataframe diferente deste poderá causar erros na coluna sentiment e isso refletirá em erros de resposta das questões.

In [ ]:
imdb_df = spark.read.csv('imdb-reviews-pt-br.csv',
                         header=True,
                         quote="\"",
                         escape="\"",
                         encoding="UTF-8")


# Questão 1

Nesta questão, você irá calcular a soma dos IDs para entradas onde o sentimento ('sentiment') é 'neg'.

### Objetivo:
- Usar a coluna 'sentiment' como chave e somar os valores da coluna 'id'.

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave e o "id" como valor do tipo inteiro

A função map irá transformar cada linha do dataframe em uma **tupla** (chave-valor), onde:
- Chave: coluna 'sentiment'
- Valor: coluna 'id' convertida para inteiro.

In [ ]:
print("Aluno: Hellen Aparecida Moura Silva")
print("RU: 3704426")

#  Definição da função map1
# map1 recebe uma linha (Row) e retorna uma tupla (sentiment, id_int)

def map1(row):
    # converte Row em dicionário para acesso seguro
    try:
        d = row.asDict()
    except Exception:

        try:
            sentiment = getattr(row, "sentiment", None)
            id_raw = getattr(row, "id", None)
            id_int = int(id_raw) if id_raw is not None else 0
            return (sentiment, id_int)
        except Exception:
            return (None, 0)

    sentiment = d.get('sentiment')
    id_raw = d.get('id', 0)
    try:
        id_int = int(id_raw)
    except (ValueError, TypeError):
        # se não for conversível, atribui 0 (evita erro no pipeline)
        id_int = 0

    return (sentiment, id_int)


## Cria funções de REDUCE:

- Criar função de reduce para somar os IDs por "sentiment".

A função reduce irá somar os valores dos IDs agrupados por chave ('sentiment').

In [ ]:
print("Aluno: Hellen Aparecida Moura Silva")
print("RU: 3704426")

#  Definição da função reduceByKey1
# recebe dois valores inteiros e retorna a soma

def reduceByKey1(x, y):
    return x + y


## Aplicação do map/reduce e visualização do resultado

Aqui, você aplicará as funções de map e reduce ao dataframe Spark para calcular os resultados. Não se esqueça de usar o método `.collect()` para visualizar os resultados.

In [ ]:
print("Aluno: Hellen Aparecida Moura Silva")
print("RU: 3704426")

#Aplicação do map/reduce e impressão do resultado
# Aplicar map e reduceByKey
resultado = imdb_df.rdd.map(map1).reduceByKey(reduceByKey1).collect()

print("---- Resultado (soma dos ids por sentiment) ----")

# Ordenar resultado por chave para apresentação consistente
for chave, soma in sorted(resultado, key=lambda x: str(x[0])):
    print(f"sentiment = {chave!s}  -> soma(id) = {soma}")

# Se quiser especificamente a soma para 'neg', exibimos abaixo:
resultado_dict = dict(resultado)
neg_sum = resultado_dict.get('neg')
print("\nSoma dos ids onde sentiment == 'neg':", neg_sum)


# Questão 2:

Nesta questão, você irá calcular a diferença no número total de palavras entre textos negativos em português e inglês.

### Objetivo:
- Contar as palavras em cada idioma (colunas 'text_pt' e 'text_en') para entradas onde o sentimento ('sentiment') é 'neg'.
- Subtrair o total de palavras em inglês do total em português.

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave de uma tupla principal e como valor uma outra tupla com a soma das palavras de cada idioma como valor.

A função map irá transformar cada linha do dataframe em uma tupla (chave-valor), onde:
- Chave: coluna 'sentiment'
- Valor: Nova tupla com:
  - Elemento 0: soma das palavras da coluna 'text_en'
  - Elemento 1: soma das palavras da coluna 'text_pt'

OU
- Chave: coluna 'sentiment'
- Valor: (soma das palavras da coluna 'text_pt') - (soma das palavras da coluna 'text_en')
  

Para contar as palavras deve-se primeiro separar os textos em uma lista de palavras para então descobrir o tamanho desta lista.
Dicas:

1. Use o método .split() e não .split(" ") de string para separar as palavras em uma lista ou use a função split(coluna de texto, regex) do pyspark com o regex igual à "[ ]+" ou "\s+"
2. Use len() para descobrir o tamanho da lista de palavras.

In [ ]:
print("Aluno: Hellen Aparecida Moura Silva")
print("RU: 3704426")
# Retorna (sentiment, (qtde_palavras_en, qtde_palavras_pt))

def map2(row):
    # tenta acessar como dicionário (Row.asDict()), fallback para getattr
    try:
        d = row.asDict()
        sentiment = d.get('sentiment')
        text_en = d.get('text_en', '') or ''
        text_pt = d.get('text_pt', '') or ''
    except Exception:
        sentiment = getattr(row, "sentiment", None)
        text_en = getattr(row, "text_en", '') or ''
        text_pt = getattr(row, "text_pt", '') or ''

    # garante strings
    if text_en is None:
        text_en = ''
    if text_pt is None:
        text_pt = ''

    # separa em palavras usando split() (trata múltiplos espaços e quebras)
    try:
        en_count = len(str(text_en).split())
    except Exception:
        en_count = 0

    try:
        pt_count = len(str(text_pt).split())
    except Exception:
        pt_count = 0

    return (sentiment, (en_count, pt_count))


## Cria funções de REDUCE:

- Criar função de reduce para somar o numero de palavras de cada texto português e inglês por "sentiment" (dependerá de como você optou por fazer sua função map2).

A função reduce irá somar os valores das quantidades de palavras agrupados por chave ('sentiment').

In [ ]:
print("Aluno: Hellen Aparecida Moura Silva")
print("RU: 3704426")
# Recebe duas tuplas (en_count, pt_count) e soma elemento a elemento

def reduceByKey2(x, y):
    # x e y são tuplas: (en_total, pt_total)
    return (x[0] + y[0], x[1] + y[1])


## Aplicação do map/reduce e visualização do resultado

1. Aplicar o map/reduce no seu dataframe spark e realizar o collect() ao final
2. Selecionar os dados referentes aos textos negativos para realizar a subtração.
3. Realizar a subtração das contagens de palavras dos textos negativos para obter o resultado final

In [ ]:
print("Aluno: Hellen Aparecida Moura Silva")
print("RU: 3704426")

# Aplica map e reduceByKey no RDD do DataFrame imdb_df
resultado = imdb_df.rdd.map(map2).reduceByKey(reduceByKey2).collect()
print("---- Soma total de palavras por sentiment (en, pt) ----")

# Imprime todos os resultados (por segurança) ordenados por sentiment
for chave, (en_total, pt_total) in sorted(resultado, key=lambda x: str(x[0])):
    print(f"sentiment = {chave!s}  -> en_total = {en_total}, pt_total = {pt_total}")

# Extrai especificamente os valores para 'neg' e calcula a diferença pt - en
resultado_dict = dict(resultado)
neg_tuple = resultado_dict.get('neg')

if neg_tuple is None:
    print("\nNão foram encontradas entradas com sentiment == 'neg'.")
else:
    en_neg, pt_neg = neg_tuple
    diff_pt_minus_en = pt_neg - en_neg
    print("\n--- Resultado pedido ---")
    print("Soma total de palavras (text_en) para sentiment == 'neg':", en_neg)
    print("Soma total de palavras (text_pt) para sentiment == 'neg':", pt_neg)
    print("Diferença (pt - en) para sentiment == 'neg':", diff_pt_minus_en)

